In [1]:
%load_ext autoreload
%autoreload 2

from config.rnn import default
from models import RNN
import numpy as np
from functional import seq
import tensorflow as tf
from tensorpack import (TrainConfig, SyncMultiGPUTrainerParameterServer as Trainer, 
                        PredictConfig, MultiProcessDatasetPredictor as Predictor,
                        SaverRestore, logger)
from tensorpack.callbacks import (ScheduledHyperParamSetter, MaxSaver, ModelSaver,
                                  DataParallelInferenceRunner as InfRunner)
from tensorpack.predict import SimpleDatasetPredictor
from tensorpack.tfutils.common import get_default_sess_config
from utils import DataManager
from utils.validation import (Accumulator, AggregateMetric, calcu_metrics)

resnet_loc = "./data/resnet_v2_101/resnet_v2_101.ckpt"
log_dir = './train_log/transfer/all_stages-g2-hidden-l10/'
logger.set_logger_dir(log_dir)

[1030 21:34:41 @logger.py:94] WRN Log directory ./train_log/transfer/all_stages-g2-hidden-l10/ exists! Please either backup/delete it, or use a new directory.
[1030 21:34:41 @logger.py:96] WRN If you're resuming from a previous run you can choose to keep it.
[1030 21:34:41 @logger.py:97] Select Action: k (keep) / b (backup) / d (delete) / n (new) / q (quit):
d
[1030 21:34:43 @logger.py:74] Argv: /home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1012/jupyter/kernel-a1e8e52c-a81c-4aa2-b00a-7afe90aaac99.json


In [2]:
config = default
ignore_restore = ['learning_rate', 'global_step']
save_name = "all-stages-max-micro-auc.tfmodel"

#### Split train set and test set

In [3]:
config.stages = [2, 3, 4, 5, 6]
config.proportion = {'train': 0.55, 'val': 0.0, 'test': 0.45}
config.annotation_number = None
dm = DataManager.from_config(config)
train_set = dm.get_train_set()
test_set = dm.get_test_set()

#### Gether data to train rnn

In [4]:
config.proportion = {'train': 0.8, 'val':0.2, 'test': 0.0}
config.stages = [2, 3, 4, 5, 6]
config.annotation_number = 10
dm = DataManager.from_dataset(train_set, test_set, config)
dm.get_num_info()

{'test': (3209, 12627), 'train': (2898, 11880), 'val': (678, 2748)}

#### Aggregate Training

In [5]:
config.weight_decay = 0.0
config.dropout_keep_prob = 0.5
config.gamma = 2
config.use_glimpse = True
config.read_time = 5
config.batch_size = 64
config.use_hidden_dense = True

threshold = 0.4
train_data = dm.get_train_stream()
val_data = dm.get_validation_stream()
model = RNN(config, is_finetuning=False)

In [6]:
tf.reset_default_graph()
train_config = TrainConfig(model=model, dataflow=train_data,
                           callbacks=[
                               ScheduledHyperParamSetter('learning_rate', [(0, 1e-4), (15, 1e-5)]),
                               InfRunner(val_data, [AggregateMetric(config.validation_metrics, threshold)],
                                         [0, 1]),
                               ModelSaver(var_collections='model_variables'),
                               MaxSaver('micro_auc', save_name),
                           ],
                           session_init=SaverRestore(
                               model_path=resnet_loc, ignore=ignore_restore),
                           max_epoch=20, tower=[0, 1])
Trainer(train_config).train()

[1030 21:34:52 @inference_runner.py:83] InferenceRunner will eval on an InputSource of size 10
[1030 21:34:53 @input_source.py:179] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...
[1030 21:34:53 @input_source.py:460] Setting up StagingArea for GPU prefetching ...
[1030 21:34:53 @training.py:41] Training a model of 2 towers
[1030 21:34:53 @training.py:92] Building graph for training tower 0 on device LeastLoadedDeviceSetter-/gpu:0...
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
IN

[1030 21:35:10 @base.py:212] Creating the session ...
[1030 21:35:13 @base.py:216] Initializing the session ...
[1030 21:35:13 @sessinit.py:116] Restoring checkpoint from ./data/resnet_v2_101/resnet_v2_101.ckpt ...
INFO:tensorflow:Restoring parameters from ./data/resnet_v2_101/resnet_v2_101.ckpt
[1030 21:35:14 @base.py:223] Graph Finalized.
[1030 21:35:14 @param.py:144] After epoch 0, learning_rate will change to 0.00010000
[1030 21:35:14 @concurrency.py:36] Starting EnqueueThread DataParallelInferenceRunner/QueueInput/input_queue ...
[1030 21:35:16 @concurrency.py:36] Starting EnqueueThread QueueInput/input_queue ...
[1030 21:35:16 @input_source.py:419] Pre-filling staging area ...
[1030 21:35:20 @base.py:257] Start Epoch 1 ...


100%|##########|45/45[01:43<00:00, 0.41it/s]

[1030 21:37:03 @base.py:267] Epoch 1 (global_step 45) finished, time:103.25 sec.



  0%|          |0/10[00:00<?,?it/s]/home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
100%|##########|10/10[00:08<00:00, 1.23it/s]


...

[1030 22:07:41 @saver.py:89] Model saved to ./train_log/transfer/all_stages-g2-hidden-l10/model-855.
[1030 22:07:41 @monitor.py:362] DataParallelInferenceRunner/QueueInput/queue_size: 49.581
[1030 22:07:41 @monitor.py:362] QueueInput/queue_size: 49.401
[1030 22:07:41 @monitor.py:362] coverage: 1.4516
[1030 22:07:41 @monitor.py:362] learning_rate: 1e-05
[1030 22:07:41 @monitor.py:362] loss/value: 0.041098
[1030 22:07:41 @monitor.py:362] macro_auc: 0.93193
[1030 22:07:41 @monitor.py:362] macro_f1: 0.69988
[1030 22:07:41 @monitor.py:362] mean_average_precision: 0.7737
[1030 22:07:41 @monitor.py:362] micro_auc: 0.94386
[1030 22:07:41 @monitor.py:362] micro_f1: 0.70469
[1030 22:07:41 @monitor.py:362] one_error: 0.25156
[1030 22:07:41 @monitor.py:362] ranking_loss: 0.062453
[1030 22:07:41 @monitor.py:362] ranking_mean_average_precision: 0.83722
[1030 22:07:41 @monitor.py:362] training_auc: 0.9392
[1030 22:07:41 @group.py:42] Callbacks took 8.377 sec in total. DataParallelInferenceRunner: 8.0

100%|##########|45/45[01:32<00:00, 0.49it/s]

[1030 22:09:13 @base.py:267] Epoch 20 (global_step 900) finished, time:92.22 sec.



100%|##########|10/10[00:08<00:00, 1.24it/s]


[1030 22:09:21 @saver.py:89] Model saved to ./train_log/transfer/all_stages-g2-hidden-l10/model-900.
[1030 22:09:22 @saver.py:158] Model with maximum 'micro_auc' saved.
[1030 22:09:22 @monitor.py:362] DataParallelInferenceRunner/QueueInput/queue_size: 49.524
[1030 22:09:22 @monitor.py:362] QueueInput/queue_size: 49.392
[1030 22:09:22 @monitor.py:362] coverage: 1.4328
[1030 22:09:22 @monitor.py:362] learning_rate: 1e-05
[1030 22:09:22 @monitor.py:362] loss/value: 0.039914
[1030 22:09:22 @monitor.py:362] macro_auc: 0.93428
[1030 22:09:22 @monitor.py:362] macro_f1: 0.70258
[1030 22:09:22 @monitor.py:362] mean_average_precision: 0.77726
[1030 22:09:22 @monitor.py:362] micro_auc: 0.94559
[1030 22:09:22 @monitor.py:362] micro_f1: 0.70824
[1030 22:09:22 @monitor.py:362] one_error: 0.225
[1030 22:09:22 @monitor.py:362] ranking_loss: 0.059616
[1030 22:09:22 @monitor.py:362] ranking_mean_average_precision: 0.84881
[1030 22:09:22 @monitor.py:362] training_auc: 0.9415
[1030 22:09:22 @group.py:42] 

#### Train a Smaller Data Set

In [5]:
test_loc_dir = 'train_log/transfer/test/D1-hidden'
model_name = 'train_log/transfer/all_stages-g2-hidden/all-stages-max-micro-auc.tfmodel'
test_save_name = 'max-training-auc.tfmodel'
ignore_restore = ['learning_rate', 'global_step', 'logits/weights', 'logits/biases', 
                  'custom_cnn/conv1/weights', 'custom_cnn/conv1/biases',
                  'custom_cnn/batch_norm1/beta', 'custom_cnn/batch_norm1/gamma',
                  'custom_cnn/batch_norm1/moving_mean', 'custom_cnn/batch_norm1/moving_variance',
                  'hidden_fc/weights', 'hidden_fc/biases']
logger.set_logger_dir(test_loc_dir)


[1029 21:12:35 @logger.py:94] WRN Log directory train_log/transfer/test/D1-hidden exists! Please either backup/delete it, or use a new directory.
[1029 21:12:35 @logger.py:96] WRN If you're resuming from a previous run you can choose to keep it.
[1029 21:12:35 @logger.py:97] Select Action: k (keep) / b (backup) / d (delete) / n (new) / q (quit):
d
[1029 21:12:37 @logger.py:74] Argv: /home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1012/jupyter/kernel-9d57e237-8682-4c42-be42-415ed30e28a8.json


In [6]:
config.weight_decay = 0.0
config.dropout_keep_prob = 0.5
config.gamma = 2
config.use_glimpse = True
config.read_time = 5
config.batch_size = 64
config.use_hidden_dense = True

threshold = 0.4
config.proportion = {'train': 1.0, 'val': 0.0, 'test': 0.0}
config.stages = [2]
config.annotation_number = 10
dm = DataManager.from_dataset(train_set, test_set, config)
train_data = dm.get_train_stream()
val_data = dm.get_test_stream()
model = RNN(config, is_finetuning=True)

In [7]:
tf.reset_default_graph()
train_config = TrainConfig(model=model, dataflow=train_data,
                           callbacks=[
                               ScheduledHyperParamSetter('learning_rate', [(0, 1e-4), (15, 1e-5)]),
                               InfRunner(val_data, [AggregateMetric(config.validation_metrics, threshold)],
                                         [0, 1]),
                               ModelSaver(var_collections='model_variables'),
                               MaxSaver('micro_auc', test_save_name),
                           ],
                           session_init=SaverRestore(
                               model_path=model_name, ignore=ignore_restore),
                           max_epoch=20, tower=[0, 1])
Trainer(train_config).train()

[1029 21:12:39 @inference_runner.py:83] InferenceRunner will eval on an InputSource of size 11
[1029 21:12:40 @input_source.py:179] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...
[1029 21:12:40 @input_source.py:460] Setting up StagingArea for GPU prefetching ...
[1029 21:12:40 @training.py:41] Training a model of 2 towers
[1029 21:12:40 @training.py:92] Building graph for training tower 0 on device LeastLoadedDeviceSetter-/gpu:0...
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
IN

100%|##########|13/13[00:35<00:00, 0.43it/s]

[1029 21:13:40 @base.py:267] Epoch 1 (global_step 13) finished, time:35.14 sec.



  0%|          |0/11[00:00<?,?it/s]/home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
100%|##########|11/11[00:10<00:00, 0.79it/s]


[1029 21:13:51 @saver.py:89] Model saved to train_log/transfer/test/D1-hidden/model-13.
[1029 21:13:51 @saver.py:158] Model with maximum 'micro_auc' saved.
[1029 21:13:51 @monitor.py:362] DataParallelInferenceRunner/QueueInput/queue_size: 50
[1029 21:13:51 @monitor.py:362] QueueInput/queue_size: 39.865
[1029 21:13:51 @monitor.py:362] coverage: 3.4134
[1029 21:13:51 @monitor.py:362] learning_rate: 0.0001
[1029 21:13:51 @monitor.py:362] loss/value: 0.18256
[1029 21:13:51 @monitor.py:362] macro_auc: 0.66507
[1029 21:13:51 @monitor.py:362] macro_f1: 0.2569
[1029 21:13:51 @monitor.py:362] mean_average_precision: 0.40976
[1029 21:13:51 @monitor.py:362] micro_auc: 0.74073
[1029 21:13:51 @monitor.py:362] micro_f1: 0.47209
[1029 21:13:51 @monitor.py:362] one_error: 0.48295
[1029 21:13:51 @monitor.py:362] ranking_loss: 0.23216
[1029 21:13:51 @monitor.py:362] ranking_mean_average_precision: 0.64654
[1029 21:13:51 @monitor.py:362] training_auc: 0.64592
[1029 21:13:51 @group.py:42] Callbacks took 1

...

100%|##########|13/13[00:26<00:00, 0.48it/s]

[1029 21:25:23 @base.py:267] Epoch 20 (global_step 260) finished, time:26.69 sec.



100%|##########|11/11[00:09<00:00, 0.85it/s]

[1029 21:25:33 @saver.py:89] Model saved to train_log/transfer/test/D1-hidden/model-260.


[1029 21:25:33 @saver.py:158] Model with maximum 'micro_auc' saved.
[1029 21:25:33 @monitor.py:362] DataParallelInferenceRunner/QueueInput/queue_size: 49.679
[1029 21:25:33 @monitor.py:362] QueueInput/queue_size: 49.581
[1029 21:25:33 @monitor.py:362] coverage: 2.8693
[1029 21:25:33 @monitor.py:362] learning_rate: 1e-05
[1029 21:25:33 @monitor.py:362] loss/value: 0.071146
[1029 21:25:33 @monitor.py:362] macro_auc: 0.8272
[1029 21:25:33 @monitor.py:362] macro_f1: 0.50828
[1029 21:25:33 @monitor.py:362] mean_average_precision: 0.5668
[1029 21:25:33 @monitor.py:362] micro_auc: 0.85402
[1029 21:25:33 @monitor.py:362] micro_f1: 0.59322
[1029 21:25:33 @monitor.py:362] one_error: 0.42614
[1029 21:25:33 @monitor.py:362] ranking_loss: 0.16158
[1029 21:25:33 @monitor.py:362] ranking_mean_average_precision: 0.70786
[1029 21:25:33 @monitor.py:362] training_auc: 0.88147
[1029 21:25:33 @group.py:42] Callbacks took 10.342 sec in total. DataParallelInferenceRunner: 9.759sec
[1029 21:25:33 @base.py:271

In [16]:
tf.get_collection(tf.GraphKeys.MODEL_VARIABLES)

[<tf.Variable 'resnet_v2_101/conv1/weights:0' shape=(7, 7, 3, 64) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/conv1/biases:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/gamma:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_mean:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_variance:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/shortcut/weights:0' shape=(1, 1, 64, 256) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/shortcut/biases:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/weights:0' shape=(1, 1, 64, 64) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/co